In [1]:
import os
import codecs
from tqdm import tqdm
import pandas as pd
import numpy as np
from utils import utils
from glob import glob
from sklearn.model_selection import train_test_split
from imp import reload
reload(utils)

<module 'utils.utils' from 'G:\\jupyter Notebook\\ccf_隐私_deepshare\\ccf-隐私-deepshare-baseline\\DBC_code\\src\\utils\\utils.py'>

In [2]:
# 采用BIOES 标注体系，把数据改成BIOES标注体系
def get_BIOES(list_data, df_label):
    ''' list_data:待标注数据
        df_label:对应数据的标签
    '''
    list_label = ['O'] * len(list_data)
    for index,d in df_label.iterrows():
#         print('d:',d)
        ID = d['ID']
        Category = d['Category']
        Pos_b = d['Pos_b']
        Pos_e = d['Pos_e']
        # 异常数据处理
        if 2162 == ID:
            Category == '前明骏女孩组合队长"'
            Pos_e = Pos_e - 1

        if Pos_b == Pos_e:
            list_label[Pos_b] = 'S_' + Category
        elif Pos_e - Pos_b == 1:
            list_label[Pos_b] = 'B_' + Category
            list_label[Pos_e] = 'E_' + Category
        else:
            # 异常数据处理
            try:
                list_label[Pos_b] = 'B_' + Category
                list_label[Pos_e] = 'E_' + Category 
                for pos_i in range(Pos_b+1,Pos_e):
                    list_label[pos_i] = 'I_' + Category 
            except:
                print('无法标注的样本：',ID)
    return [(ID,data,label) for data,label in zip(list_data,list_label)]

In [3]:
def get_list_data(data_path):
    with open(data_path,'r',encoding='utf-8') as fin:
        line = ''.join(fin.readlines())
        list_data = list(line)
    return list_data
data = get_list_data(r'G:\jupyter Notebook\ccf_隐私_deepshare\ccf-隐私-deepshare-baseline\DBC_code\data\train_data\2162.txt')
print(data)

['公', '司', '：', '海', '润', '影', '视', '电', '影', '作', '品', '：', '《', '公', '主', '的', '诱', '惑', '》', '《', '恋', '爱', '三', '万', '英', '尺', '》', '《', '囍', '上', '加', '喜', '》', '音', '乐', '作', '品', '：', '同', '名', '专', '辑', '《', 'S', '公', '主', '》', '经', '纪', '人', '莲', '莲', '：', '1', '3', '9', '1', '0', '2', '2', '0', '8', '9', '2', '，', '歌', '手', '演', '员', '前', '明', '骏', '女', '孩', '组', '合', '队', '长']


In [4]:
# 将/data/train_data/,/data/train_label/文件下数据标注出来，并生成对应的文件
def get_list_data(data_path):
    with open(data_path,'r',encoding='utf-8') as f:
        line = ''.join(f.readlines())
        list_data = list(line)
    return list_data

def get_data_label(path_dir_data,path_dir_label):
    
    # 训练数据集
    list_BIOES = []
    # 获取训练集 ID
    list_data_ID = [file_name.split('.')[0] for file_name in os.listdir(path_dir_data) if '.txt' in file_name]
    list_label_ID = [file_name.split('.')[0] for file_name in os.listdir(path_dir_label) if '.csv' in file_name]
    
    # 循环训练集 ID
    for ID in tqdm(set(list_data_ID) & set(list_label_ID)):
        # 获取词和标签地址
        data_path = os.path.join(path_dir_data,ID+'.txt')
        label_path = os.path.join(path_dir_label,ID+'.csv')
#         print('data_path:',data_path)
#         print('label_path:',label_path)
        # 获取词和标签数据
        list_data = get_list_data(data_path)
        df_label = pd.read_csv(label_path)
        
        # BIOES 进行标注
        BIOES = get_BIOES(list_data, df_label)
        
        # 加入训练数据集
        list_BIOES += [BIOES]
        
    return list_BIOES

In [5]:
train_dir_data = r'../data/ext_data/ext_train_data/'
train_dir_label = r'../data/ext_data/ext_label/'
list_BIOES_train = get_data_label(train_dir_data,train_dir_label)
list_BIOES_train[0]

FileNotFoundError: [WinError 3] 系统找不到指定的路径。: '../data/ext_data/ext_label/'

In [6]:
# 数据集划分，把数据标注成BIOES格式
# symbol = ['。','!','！','?','？','﹗']
# 生成样本的参数,样本划分的标点符号
symbol = ['？','⋯','…','﹗']
max_sent_length = 250
max_input_length = 300   

data_lst = glob('../data/train_data/*.txt')
train_lst,val_lst = train_test_split(data_lst,test_size=0.1,random_state=0)
print('train:{}'.format(len(train_lst)))
print('val:{}'.format(len(val_lst)))

train:2263
val:252


In [7]:
label_dir = '../data/train_label'
# 训练集
for data_path in tqdm(train_lst):
    file_name = os.path.basename(data_path).split('.')[0]
    label_name = file_name+'.csv'
    label_path = os.path.join(label_dir,label_name)
#     print('label_path:',label_path)
    output_path = '../data/train_new_bioes'
    output_file = file_name
    utils.gen_BIOES_data(label_path, data_path, output_path, output_file,symbol,max_sent_length,max_input_length)
# 验证集
for data_path in tqdm(val_lst):
    file_name = os.path.basename(data_path).split('.')[0]
    label_name = file_name+'.csv'
    label_path = os.path.join(label_dir,label_name)
    output_path = '../data/val_new_bioes'
    output_file = file_name
    utils.gen_BIOES_data(label_path, data_path, output_path, output_file,symbol,max_sent_length,max_input_length)

100%|██████████| 252/252 [00:11<00:00, 22.18it/s]


## 数据集合并成txt

In [6]:
# 训练集合并
# w_path = "../data/test_bioes.txt"
w_path = '../data/train_aug_bioes/train_aug_bioes.txt' 
path_dir = "../data/train_aug_bioes/train_bioes/"
for file in os.listdir(path_dir):
    path = os.path.join(path_dir, file)
    if not file.endswith(".txt"):  
        continue
    q_list = []
    print("开始读取文件:%s" % file)
    with open(path, "r", encoding="utf-8") as f:
        line = f.readline()
        line = line.strip("\n\r")
#         while line != "END O":
        while line != 'D	O':
            q_list.append(line)
            line = f.readline()
            line = line.strip("\n\r")
    print("开始写入文本%s" % w_path)
    with codecs.open(w_path, "a", encoding="utf-8") as f:
        for item in q_list:
            if item.__contains__('\ufeff'):
                print("===============")
            f.write('%s\n' % item)
        # 对于每个文本结束，都写个换行符，这样做有什么作用么，还是仅仅为了方便查找问题？             
        f.write('\n')
    f.close()

开始读取文件:aug-0-0.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-10.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-100.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1000.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1001.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1002.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1003.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1004.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1005.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1006.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1007.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1008.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1009.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开

开始读取文件:aug-0-1140.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1141.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1142.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1143.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1144.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1145.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1146.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1147.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1148.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1149.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-115.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1150.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1151.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1152.txt
开始写入文本../data/train_aug_bioes/train_aug_bio

开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1274.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1275.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1276.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1277.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1278.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1279.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-128.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1280.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1281.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1282.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1283.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1284.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1285.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-12

开始读取文件:aug-0-1411.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1412.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1413.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1414.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1415.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1416.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1417.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1418.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1419.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-142.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1420.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1421.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1422.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1423.txt
开始写入文本../data/train_aug_bioes/train_aug_bio

开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1551.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1552.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1553.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1554.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1555.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1556.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1557.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1558.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1559.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-156.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1560.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1561.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-1562.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-0-15

...

开始读取文件:aug-2-605.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-606.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-607.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-608.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-609.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-61.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-610.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-611.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-612.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-613.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-614.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-615.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-616.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-617.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:

开始读取文件:aug-2-726.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-727.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-728.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-729.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-73.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-730.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-731.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-732.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-733.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-734.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-735.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-736.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-737.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-738.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:

开始读取文件:aug-2-830.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-831.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-832.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-833.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-834.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-835.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-836.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-837.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-838.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-839.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-84.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-840.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-841.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-842.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:

开始读取文件:aug-2-943.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-944.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-945.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-946.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-947.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-948.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-949.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-95.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-950.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-951.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-952.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-953.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-954.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:aug-2-955.txt
开始写入文本../data/train_aug_bioes/train_aug_bioes.txt
开始读取文件:

KeyboardInterrupt: 

In [83]:
# 验证集合并
w_path = "../data/val_bioes.txt"
for file in os.listdir('../data/val_new_bioes/'):
    path = os.path.join("../data/val_new_bioes", file)
    if not file.endswith(".txt"):  
        continue
    q_list = []

    with codecs.open(path, "r", encoding="utf-8") as f:
        line = f.readline()
        line = line.strip("\n\r")
        while line != "END O":
            q_list.append(line)
            line = f.readline()
            line = line.strip("\n\r")
    
    with codecs.open(w_path, "a", encoding="utf-8") as f:
        for item in q_list:
            if item.__contains__('\ufeff'):
                print("===============")
            f.write('%s\n' % item)
        f.write('\n')
    f.close()